# ChronoTick 2: TimesFM 2.5 Fine-Tuning

Fine-tune Google TimesFM 2.5 (200M) on univariate clock drift data.
FT is strictly univariate; sensor covariates are only used at inference
via XReg (ridge regression wrapper).

## Experiments
- E1: Univariate FT
- E2: FT + XReg at inference (compare to FT-only)
- E3: Per-machine vs combined training

## Training Mode
Set `TRAINING_MODE` to "combined" or "per_machine".

In [ ]:
# === Environment Setup ===
import os, subprocess, sys

IN_COLAB = "COLAB_GPU" in os.environ or os.path.exists("/content")

if IN_COLAB:
    REPO_DIR = "/content/sensor-collector"
    REPO_URL = "https://github.com/JaimeCernuda/sensor-collector.git"
    GITHUB_TOKEN = None
    try:
        from google.colab import userdata
        GITHUB_TOKEN = userdata.get("GITHUB_TOKEN")
    except Exception:
        print("WARNING: GITHUB_TOKEN not available")
    auth_url = (
        f"https://{GITHUB_TOKEN}@github.com/JaimeCernuda/sensor-collector.git"
        if GITHUB_TOKEN
        else REPO_URL
    )
    if os.path.exists(REPO_DIR):
        subprocess.run(["git", "-C", REPO_DIR, "remote", "set-url", "origin", auth_url], check=True)
        subprocess.run(["git", "-C", REPO_DIR, "fetch", "-q", "origin"], check=True)
        subprocess.run(["git", "-C", REPO_DIR, "reset", "--hard", "origin/main"], check=True)
    else:
        subprocess.run(["git", "clone", "-q", auth_url, REPO_DIR], check=True)
    subprocess.run(["git", "-C", REPO_DIR, "config", "user.name", "Colab Runner"], check=True)
    subprocess.run(["git", "-C", REPO_DIR, "config", "user.email", "colab@chronotick.dev"], check=True)
    subprocess.run(["pip", "install", "-q", "-e", f"{REPO_DIR}/tick2/"], check=True)
    tick2_src = f"{REPO_DIR}/tick2/src"
    if tick2_src not in sys.path:
        sys.path.insert(0, tick2_src)

    # Always mount Drive — needed for checkpoint persistence (models too large for git)
    from google.colab import drive
    drive.mount("/content/drive")

    # Data: prefer repo copy, fall back to Drive
    DATA_DIR = f"{REPO_DIR}/sensors/data"
    if not os.path.isdir(f"{DATA_DIR}/24h_snapshot"):
        DATA_DIR = "/content/drive/MyDrive/chronotick2/data"

    RESULTS_DIR = f"{REPO_DIR}/tick2/notebooks/output/03"
else:
    GITHUB_TOKEN = None
    DATA_DIR = None
    RESULTS_DIR = os.path.join(
        os.path.dirname("__file__") if "__file__" in dir() else ".", "output", "03"
    )

DEVICE_DIR_MAP = {"cuda": "gpu", "cpu": "cpu"}


def checkpoint_push(label):
    """Git add, commit, and push notebook 03d results."""
    if not IN_COLAB:
        return
    try:
        subprocess.run(
            ["git", "-C", REPO_DIR, "add", "tick2/notebooks/output/03/"],
            check=True, capture_output=True,
        )
        status = subprocess.run(
            ["git", "-C", REPO_DIR, "status", "--porcelain", "tick2/notebooks/output/03/"],
            capture_output=True, text=True,
        )
        if not status.stdout.strip():
            return
        subprocess.run(
            ["git", "-C", REPO_DIR, "commit", "-m",
             f"results: notebook 03d timesfm {label} ({device_label})"],
            check=True, capture_output=True,
        )
        if GITHUB_TOKEN:
            subprocess.run(
                ["git", "-C", REPO_DIR, "push"],
                check=True, capture_output=True, timeout=60,
            )
            print(f"  [CHECKPOINT] Pushed {label}")
    except Exception as e:
        print(f"  [CHECKPOINT WARNING] {e}")


print(f"Environment: {'Colab' if IN_COLAB else 'Local'}")

In [ ]:
# === Install TimesFM Dependencies ===
if IN_COLAB:
    subprocess.run(
        "git clone -q --depth 1 https://github.com/google-research/timesfm /content/timesfm".split(),
        capture_output=True,
    )
    subprocess.run(
        "touch /content/timesfm/src/timesfm/timesfm_2p5/__init__.py".split(),
        check=True,
    )
    subprocess.run(
        ["pip", "install", "-q", "/content/timesfm[torch]"],
        check=True,
    )

import timesfm
assert hasattr(timesfm, "TimesFM_2p5_200M_torch"), "TimesFM 2.5 not available"
print("timesfm ready")

In [ ]:
# === Imports, Config & Training Mode ===
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from pathlib import Path

from tick2.data.preprocessing import TARGET_COL, load_all
from tick2.finetuning.base import FineTuneConfig
from tick2.finetuning.data_prep import prepare_datasets
from tick2.finetuning.timesfm_ft import finetune_timesfm, load_finetuned_timesfm
from tick2.finetuning.evaluate import (
    evaluate_finetuned,
    load_zero_shot_baselines,
    compare_ft_vs_zero_shot,
)
from tick2.utils.gpu import clear_gpu_memory

sns.set_theme(style="whitegrid", font_scale=1.1)

# --- User Configuration ---
TRAINING_MODE = "combined"  # "combined" or "per_machine"
DEVICE_OVERRIDE = None       # None = auto-detect, "cuda", or "cpu"
FORCE_RETRAIN = False        # Set True to retrain even if cached results exist

# --- Derived settings ---
device = DEVICE_OVERRIDE or ("cuda" if torch.cuda.is_available() else "cpu")
device_label = DEVICE_DIR_MAP.get(device, device)
config = FineTuneConfig(
    context_length=1024,
    prediction_length=96,
    max_covariates=30,
    seed=42,
)
print(f"Device: {device}, Mode: {TRAINING_MODE}")

In [ ]:
# === Load and Prepare Data ===
data_dir = Path(DATA_DIR) if DATA_DIR else None
prepared = prepare_datasets(config, data_dir=data_dir)
for name, p in prepared.items():
    print(
        f"  {name:16s}: train={len(p.split.train)}, "
        f"val={len(p.split.val)}, test={len(p.split.test)}"
    )

In [ ]:
# === Fine-Tune TimesFM 2.5 ===
from tick2.utils.colab import save_checkpoint_to_drive, load_checkpoint_from_drive, setup_training_log

output_base = Path(RESULTS_DIR)
ft_output_dir = output_base / "timesfm_ft" / TRAINING_MODE
device_results_dir = output_base / device_label

# Persist training logs to disk (epoch losses, early stopping, errors)
log_path = setup_training_log(ft_output_dir)
print(f"Training log: {log_path}")

cached_path = device_results_dir / f"timesfm-2.5-ft_{TRAINING_MODE}.csv"

# Check for existing Drive checkpoint (resume after disconnect)
drive_model_name = f"timesfm_ft/{TRAINING_MODE}/best_model.pt"
ckpt_local = ft_output_dir / "combined" / "best_model.pt"
if not cached_path.exists() and not FORCE_RETRAIN and not ckpt_local.exists():
    resumed = load_checkpoint_from_drive(
        model_name=drive_model_name,
        local_path=str(ckpt_local),
    )
    if resumed:
        print(f"[RESUMED] Loaded checkpoint from Drive: {resumed}")

if cached_path.exists() and not FORCE_RETRAIN:
    print(f"[CACHED] {cached_path}")
elif ckpt_local.exists() and not FORCE_RETRAIN:
    print(f"[CACHED] Checkpoint exists at {ckpt_local}, skipping training")
else:
    clear_gpu_memory()
    ft_results = finetune_timesfm(
        prepared=prepared,
        config=config,
        output_dir=str(ft_output_dir),
        training_mode=TRAINING_MODE,
        learning_rate=1e-4,
        num_epochs=50,
        batch_size=64,
        context_length=128,
        horizon_length=32,
    )
    for r in ft_results:
        print(f"  {r.machine}: {r.training_time_s:.1f}s")

    # Save checkpoint to Drive for persistence (~800MB for TimesFM)
    save_checkpoint_to_drive(
        local_path=ft_output_dir / "combined",
        model_name=f"timesfm_ft/{TRAINING_MODE}",
    )

    checkpoint_push("finetuning")

In [ ]:
# === Evaluate Fine-Tuned Model ===
from tick2.models.timesfm import TimesFMWrapper

if cached_path.exists() and not FORCE_RETRAIN:
    ft_eval_df = pd.read_csv(cached_path)
    print(f"Loaded cached evaluation: {len(ft_eval_df)} rows")
else:
    ckpt_path = ft_output_dir / "combined" if TRAINING_MODE == "combined" else ft_output_dir
    ft_model = load_finetuned_timesfm(str(ckpt_path))

    ft_wrapper = TimesFMWrapper(model_name="timesfm-2.5-ft")
    ft_wrapper._model = ft_model
    ft_wrapper._device = device

    ft_eval_df = evaluate_finetuned(
        model=ft_wrapper,
        prepared=prepared,
        config=config,
        training_mode=f"ft_{TRAINING_MODE}",
    )
    device_results_dir.mkdir(parents=True, exist_ok=True)
    ft_eval_df.to_csv(cached_path, index=False)
    checkpoint_push("evaluation")

print(f"Mean MAE: {ft_eval_df['mae'].mean():.4f}")

In [ ]:
# === Load Zero-Shot Baselines ===
zs_dir = output_base.parent / "output" / "02"
zs_results = load_zero_shot_baselines(zs_dir, model_name="timesfm-2.5")
print(f"Zero-shot: {len(zs_results)} rows")

In [ ]:
# === Compare Fine-Tuned vs Zero-Shot ===
comparison_df = compare_ft_vs_zero_shot(ft_eval_df, zs_results)

# Compute per-machine improvement
if not comparison_df.empty and len(comparison_df["training_mode"].unique()) > 1:
    pivot = comparison_df.pivot_table(
        values="mae",
        index=["machine", "context_length", "horizon"],
        columns="training_mode",
        aggfunc="mean",
    )
    zs_col = "zero_shot"
    ft_col = f"ft_{TRAINING_MODE}"
    if zs_col in pivot.columns and ft_col in pivot.columns:
        pivot["improvement_pct"] = (
            (pivot[zs_col] - pivot[ft_col]) / pivot[zs_col] * 100
        )
        print("\n=== Per-Config Improvement ===")
        print(pivot.to_string())
        print(f"\nMean improvement: {pivot['improvement_pct'].mean():.1f}%")
else:
    print("No zero-shot baselines found for comparison.")
    print("Run notebook 02 first to generate TimesFM 2.5 zero-shot results.")

display(comparison_df)

In [ ]:
# === Visualizations ===
fig_dir = output_base / "figures"
fig_dir.mkdir(parents=True, exist_ok=True)

# --- 1. MAE Comparison: Fine-Tuned vs Zero-Shot ---
if not comparison_df.empty and len(comparison_df["training_mode"].unique()) > 1:
    fig, ax = plt.subplots(figsize=(10, 5))
    uni_cmp = comparison_df[~comparison_df["with_covariates"]]
    sns.barplot(
        data=uni_cmp, x="machine", y="mae",
        hue="training_mode", ax=ax,
    )
    ax.set_ylabel("MAE (ppm)")
    ax.set_title("TimesFM 2.5: Fine-Tuned vs Zero-Shot MAE")
    ax.legend(title="Mode")
    plt.tight_layout()
    fig.savefig(fig_dir / "timesfm_ft_vs_zs_mae.png", dpi=150, bbox_inches="tight")
    plt.show()

# --- 2. Training Loss Curve ---
if "ft_results" in dir() and ft_results:
    fig, axes = plt.subplots(1, len(ft_results), figsize=(6 * len(ft_results), 4), squeeze=False)
    for i, r in enumerate(ft_results):
        ax = axes[0][i]
        if r.train_loss:
            ax.plot(r.train_loss, label="Train", alpha=0.8)
        if r.val_loss:
            ax.plot(r.val_loss, label="Val", alpha=0.8)
            ax.axvline(r.best_epoch, color="red", ls="--", alpha=0.5, label=f"Best epoch ({r.best_epoch})")
        ax.set_xlabel("Epoch")
        ax.set_ylabel("Loss")
        ax.set_title(f"Training Loss: {r.machine}")
        ax.legend()
    plt.tight_layout()
    fig.savefig(fig_dir / "timesfm_ft_loss_curves.png", dpi=150, bbox_inches="tight")
    plt.show()

# --- 3. MAE by Machine and Horizon (FT only) ---
if not ft_eval_df.empty:
    fig, ax = plt.subplots(figsize=(10, 5))
    uni_ft = ft_eval_df[~ft_eval_df["with_covariates"]]
    if not uni_ft.empty:
        sns.barplot(
            data=uni_ft, x="machine", y="mae",
            hue="horizon", ax=ax,
        )
        ax.set_ylabel("MAE (ppm)")
        ax.set_title(f"Fine-Tuned TimesFM 2.5 MAE by Horizon ({TRAINING_MODE})")
        ax.legend(title="Horizon")
    plt.tight_layout()
    fig.savefig(fig_dir / "timesfm_ft_mae_by_horizon.png", dpi=150, bbox_inches="tight")
    plt.show()

print(f"Figures saved to: {fig_dir}")

In [ ]:
# === Export Results ===
export_dir = output_base / device_label
export_dir.mkdir(parents=True, exist_ok=True)

# CSV: fine-tuned evaluation
ft_csv_path = export_dir / f"timesfm-2.5-ft_{TRAINING_MODE}.csv"
ft_eval_df.to_csv(ft_csv_path, index=False)
print(f"FT results CSV: {ft_csv_path}")

# CSV: combined comparison (if baselines available)
if not comparison_df.empty:
    cmp_csv_path = export_dir / f"timesfm_ft_vs_zs_{TRAINING_MODE}.csv"
    comparison_df.to_csv(cmp_csv_path, index=False)
    print(f"Comparison CSV: {cmp_csv_path}")

# LaTeX summary table
if not comparison_df.empty and len(comparison_df["training_mode"].unique()) > 1:
    latex_summary = comparison_df.groupby(["machine", "training_mode"]).agg(
        mae_mean=("mae", "mean"),
        rmse_mean=("rmse", "mean"),
        inference_ms=("inference_ms", "mean"),
    ).round(4)
    latex_path = export_dir / f"timesfm_ft_summary_{TRAINING_MODE}.tex"
    latex_summary.to_latex(str(latex_path))
    print(f"LaTeX table:    {latex_path}")
    print("\n" + latex_summary.to_string())

In [ ]:
# === Final Push ===
if IN_COLAB:
    import os
    os.chdir(REPO_DIR)
    subprocess.run(["git", "add", "tick2/notebooks/output/03/"], check=True)
    status = subprocess.run(
        ["git", "status", "--porcelain", "tick2/notebooks/output/03/"],
        capture_output=True, text=True,
    )
    if status.stdout.strip():
        subprocess.run(
            ["git", "commit", "-m",
             f"results: notebook 03d timesfm final outputs ({device_label})"],
            check=True,
        )
        if GITHUB_TOKEN:
            subprocess.run(["git", "push"], check=True)
            print("Pushed final outputs to GitHub.")
        else:
            print("Committed locally but GITHUB_TOKEN not set.")
    else:
        print("No new outputs to commit (checkpoints already pushed).")
else:
    print(f"Local run. Outputs saved to: {output_base}")
    print("Run 'git add tick2/notebooks/output/03/ && git commit && git push' to share.")